
# 第9章 もっと入力、もっと出力

```zsh
$ghc --make capslocker.hs 
[1 of 1] Compiling Main             ( capslocker.hs, capslocker.o )
Linking capslocker ...

$./capslocker < haiku.txt   
I'M A LIL' TEAPOT
WHAT'S WITH THAT AIRPLANE FOOD, HUH?
IT'S SO SMALL, TASTELESS
capslocker: <stdin>: hGetLine: end of file
```

In [1]:
shortLinesOnly :: String -> String
shortLinesOnly = unlines . filter (\line -> length line < 10) . lines

In [2]:
:t unlines

unlines :: [String] -> String

In [3]:
main = interact shortLinesOnly

In [1]:
respondPalindromes :: String -> String
respondPalindromes = unlines . map (\xs -> if isPal xs then "palindrome" else "not a palindrome") . lines

isPal :: String -> Bool
isPal xs = xs == reverse xs

In [2]:
import System.IO

main = do
    handle <- openFile "baabaa.txt" ReadMode
    contents <- hGetContents handle
    putStr contents
    hClose handle

In [3]:
main

Baa, baa, black sheep,
Have you any wool?
Yes, sir, yes, sir,
Three bags full;

In [4]:
:t openFile

openFile :: FilePath -> IOMode -> IO Handle

In [5]:
:t FilePath

Not in scope: data constructor `FilePath'

In [6]:
:k FilePath

FilePath :: *

In [7]:
import System.IO

main = do
    withFile "baabaa.txt" ReadMode $ \handle -> do
        contents <- hGetContents handle
        putStr contents
        

Line 1: Redundant do
Found:
do withFile "baabaa.txt" ReadMode $
     \ handle ->
       do contents <- hGetContents handle
          putStr contents
Why not:
withFile "baabaa.txt" ReadMode $
  \ handle ->
    do contents <- hGetContents handle
       putStr contents

In [8]:
main

Baa, baa, black sheep,
Have you any wool?
Yes, sir, yes, sir,
Three bags full;

次回  

### ブラケットの時間

デュエルスタンバイ